# cmor dev

In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import xarray as xr

import cordex as cx
from cordex import cmor

In [13]:
# cmor.set_options(exit_control="CMOR_EXIT_ON_MAJOR")


def test_cmorizer_mon():
    ds = cx.tutorial.open_dataset("remo_EUR-11_TEMP2_mon")
    eur11 = cx.cordex_domain("EUR-11")
    ds = ds.assign_coords({"lon": eur11.lon, "lat": eur11.lat})
    filename = cmor.cmorize_variable(
        ds,
        "tas",
        mapping_table={"tas": {"varname": "TEMP2"}},
        cmor_table=cx.tables.cmip6_cmor_table("CMIP6_Amon"),
        dataset_table=cx.tables.cordex_cmor_table("CORDEX_remo_example"),
        grids_table=cx.tables.cmip6_cmor_table("CMIP6_grids"),
        CORDEX_domain="EUR-11",
        time_units=None,
        allow_units_convert=True,
    )
    return filename

In [22]:
cx.tutorial.open_dataset("remo_EUR-11_TEMP2_mon").TEMP2.attrs

{'long_name': '2m temperature',
 'units': 'K',
 'code': 167,
 'leveltype': 105,
 'grid_mapping': 'rotated_latitude_longitude'}

In [28]:
f = test_cmorizer_mon()

/Users/buntemey/python/packages/py-cordex/cordex/cmor/cmor.py:330: UserWarning: time units are set to default: days since 1949-12-01T00:00:00
  
/Users/buntemey/python/packages/py-cordex/cordex/cmor/cmor.py:512: UserWarning: adding time bounds
  
/Users/buntemey/python/packages/py-cordex/cordex/core/domain.py:632: UserWarning: Order of vertices has changed since v0.3.2 to CF Conventions, see https://github.com/euro-cordex/py-cordex/issues/34
  warn(


TypeError: argument 2 must be str, not Unit

In [17]:
ds = xr.open_dataset(f)
ds

<xarray.Dataset>
Dimensions:                     (time: 12, bnds: 2, rlat: 412, rlon: 424,
                                 vertices: 4)
Coordinates:
  * time                        (time) datetime64[ns] 2000-01-15 ... 2000-12-15
  * rlat                        (rlat) float64 -23.38 -23.27 ... 21.72 21.83
  * rlon                        (rlon) float64 -28.38 -28.27 ... 18.04 18.15
    latitude                    (rlat, rlon) float64 ...
    longitude                   (rlat, rlon) float64 ...
    height                      float64 ...
Dimensions without coordinates: bnds, vertices
Data variables:
    time_bnds                   (time, bnds) datetime64[ns] ...
    rotated_latitude_longitude  int32 ...
    vertices_latitude           (rlat, rlon, vertices) float64 ...
    vertices_longitude          (rlat, rlon, vertices) float64 ...
    tas                         (time, rlat, rlon) float32 ...
Attributes: (12/53)
    CORDEX_domain:           EUR-11
    Conventions:             CF-1.7 CMIP-6.2
    activity_id:             CORDEX
    branch_method:           standard
    branch_time_in_child:    59400.0
    branch_time_in_parent:   59400.0
    ...                      ...
    title:                   REMO2-0-0 output prepared for CMIP6
    tracking_id:             hdl:21.14100/062e926c-654f-4a96-b9ab-5d0fc9405ef2
    variable_id:             tas
    variant_label:           r1i1p1f1
    license:                 CMIP6 model data produced by Lawrence Livermore ...
    cmor_version:            3.7.0